In [ ]:
import subprocess
import matplotlib
import mglearn
import numpy
import pandas
import pdfminer
import sklearn
import sys
from io import StringIO
#Python LDA Visualization Libraries
import pyLDAvis
import pyLDAvis.sklearn
import PyPDF2
# pdfminer
from pdfminer.converter import PDFPageAggregator
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.layout import LTTextBoxHorizontal
from pdfminer.pdfdevice import PDFDevice
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.pdftypes import resolve1
#Relevant Analysis Librariess
import re
import numpy as np
import pandas as pd
import mglearn as mg
#Relevant Modeling Libraries
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
%pylab
%matplotlib inline

In [2]:
#Creating a Function to Read the PDF Document and convert into Text
def convert_pdf_to_text(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos = set()
    
    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password, caching=caching, check_extractable=True):
        interpreter.process_page(page)
        
    text = retstr.getvalue()
    
    fp.close()
    device.close()
    retstr.close()
    
    return text
#Reading the PDF Document and saving as lone
lone=convert_pdf_to_text('C:/Users/matth/Documents/Jobs/Travelers/NLP/Automated_Doc_Summarization_NLP/Claim_Req_Travelers_McDonald_5_14.pdf')


In [3]:
# Gets inputs rb
path='C:/Users/matth/Documents/Jobs/Travelers/NLP/Automated_Doc_Summarization_NLP/Claim_Req_Travelers_McDonald_5_14.pdf'
fp = open(path, 'rb')
parser = PDFParser(fp)
doc = PDFDocument(parser)
fields = resolve1(doc.catalog['AcroForm'])['Fields']

inps=[]
for i in fields:
    field = resolve1(i)
    name, value = field.get('T'), field.get('V')
    inps.append('{0}: {1}'.format(name, value))
    
inf=[]
ini=[]
for i in fields:
    field=resolve1(i)
    name,value=field.get('T'),field.get('V')
    inf.append(name)
    ini.append(value)

In [ ]:
# Topic Modeling
# Fitting Count Vectorizer on the document with Stop Words
vect=CountVectorizer(ngram_range=(1,1),stop_words='english')
dtm = vect.fit_transform(inps)

#Converting the Document Term Matrix from Count Vectorizer into a Pandas Dataframe
dfm=pd.DataFrame(dtm.toarray(), columns=vect.get_feature_names())
dfm

In [ ]:
#Fitting the Latent Dirichlet Allocation Model on the Document Term Matrix
lda = LatentDirichletAllocation(n_components=5)
lda_dtf = lda.fit_transform(dtm)
#Latent Dirichlet Allocation Model
lda_dtf

In [7]:
# Topic Extracting
#Extracting 5 Topics from LDA and the most common words in each topic
sorting = np.argsort(lda.components_)[:, ::-1]
features = np.array(vect.get_feature_names())

mg.tools.print_topics(topics=range(5), feature_names=features, sorting=sorting, topics_per_chunk=5, n_words=15)

topic 0       topic 1       topic 2       topic 3       topic 4       
--------      --------      --------      --------      --------      
date          paid          signature     claim         claimant      
05            00            yes           suit          type          
2021          defense       mcdonald      james         project       
13            costs         joseph        mcgoo         joseph        
defendants    total         representativetravelers     mcdonald      
2020          date          authorized    settlement    spine         
proposed      court         producer      number        nm            
description   claimant      electronic    individual    albuquerque   
laser         alleged       settlement    closed        location      
wrongful      description   agency        arrested      services      
act           spine         printed       safety        professional  
18            injury        demand        supplied      case          
14    

In [8]:
#Sentences within the Topic Model 1
topic_0 = np.argsort(lda_dtf[:,0])[::-1]
for i in topic_0[:5]:
    print(f".".join(inps[i].split(f".")[:2]) + f".\n")

b'Description of the alleged wrongful act': b"Defendants attempted to disintegrate claimant's spine with high powered laser beams.".

b'Description of the case and events - 1': b'On Wed. 5/13/2020, the defendants utilized extraterrestrial engineered laser'.

b'Wrongful act Date': b'05/13/2021'.

b'Proposed Expiration Date': b'05/18/2021'.

b'Proposed Effective Date': b'05/18/2020'.



In [9]:
#Senteces within the Topic Model 2
topic_1 = np.argsort(lda_dtf[:,1])[::-1]
for i in topic_1[:5]:
    print(f".".join(inps[i].split(f".")[:2]) + f".\n")

b'Description of the type and extent of alleged injury': b"Claimant's spine was broken in thoracic, lumbar, and sacral regions.".

b"Insurer's loss reserve": b'3000000.00'.

b'Defense costs paid to date': b'0.00'.

b'Total defense costs paid': None.

b'Total indemnity paid': None.



In [12]:
#Senteces within the Topic Model 3
topic_2 = np.argsort(lda_dtf[:,2])[::-1]
for i in topic_2[:5]:
    print(f".".join(inps[i].split(f".")[:2]) + f".\n")

b'Electronic Signature - Authorized Representative': /'Yes'.

b'Authorized Representative Signature': b'Joseph McDonald'.

b'Authorized Representative Name - Printed': b'Joseph McDonald'.

b"Claimant's settlement demand": b'1000000.00'.

b'Electronic Signature - Producer': /'Yes'.



In [13]:
#Senteces within the Topic Model 4
topic_3 = np.argsort(lda_dtf[:,3])[::-1]
for i in topic_3[:5]:
    print(f".".join(inps[i].split(f".")[:2]) + f".\n")

b'Description of Remedial measures': b'Defendents and extraterrestrials who supplied laser cannons have been arrested by the Intergalactic Safety Force (ISF).'.

b'Name of individual - 2': b'Sally James Temple, Mark McGoo, James McGoo, Matt McGee'.

b'Travelers Policy Number': b'6JUB-1E112'.

b"Defendant's offer for settlement": b'7000000'.

b'State Producer License No': b'625864115'.



In [14]:
#Senteces within the Topic Model 5
topic_4 = np.argsort(lda_dtf[:,4])[::-1]
for i in topic_4[:5]:
    print(f".".join(inps[i].split(f".")[:2]) + f".\n")

b'Description of the case and events - 2': b"beam cannons to attack claimant's spine. Claimant defended self and teleported defendents to Topeka, KS.

b'Name , type and location of project - 1': b'The Great Project, building construction, Albuquerque, NM'.

b'Type of professional services - 2': b'Spine adjustment, Albuquerque, NM'.

b'Name of Individual - 1': b'Joseph McDonald, Frank McDugle, '.

b'Proposed Named Insured': b'Joseph McDonald'.



In [15]:
# Topic Visualiization
zit=pyLDAvis.sklearn.prepare(lda,dtm,vect)

pyLDAvis.display(zit)